<a href="https://colab.research.google.com/github/MrScratch123/Lyrics-Predictor/blob/main/Shipathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def clean_text(text):
    return text

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

nltk.download('punkt_tab')
nltk.download('stopwords')

#Edit this file path to whatever you want before running this cell
# drive.mount('/content/drive')
save_path = '/content/cleaned_data.csv'

def data_cleaner(file_path):
  data = pd.read_csv(file_path)
  data['cleaned_lyrics'] = data['Lyrics'].apply(clean_text)
  data = data.drop(columns = ['Song', 'Song year', 'Track_id', 'Lyrics',  'Artist'])
  return data



cleaned_df = data_cleaner('/content/test.csv')
#print(cleaned_df)
cleaned_df.to_csv(save_path,index = False)




print(f"File successfully saved to : {save_path}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


File successfully saved to : /content/cleaned_data.csv


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
import pandas as pd

lyric_data = pd.read_csv(save_path)

corpus = lyric_data['cleaned_lyrics']

tfidf_vec = TfidfVectorizer()
tfidf_lib = tfidf_vec.fit_transform(corpus).toarray()

print("Library TF-IDF (first 2 rows):\n", tfidf_lib[:2])




Library TF-IDF (first 2 rows):
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

corpus = lyric_data['cleaned_lyrics']

doc_embeddings = model.encode(corpus)

X = np.array(doc_embeddings)


In [ ]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA(n_components = 500)
X = pca.fit_transform(np.array(tfidf_lib))
#X = np.array(tfidf_lib)

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

#X = np.array(tfidf_lib)

mp = {'Country' : 0,  'Electronic' : 1,  'Folk' : 2, 'Hip-Hop' : 3, 'Indie' : 4, 'Jazz': 5, 'Metal' : 6, 'Pop' : 7, 'R&B' : 8,  'Rock' : 9}

gen = []
for i in lyric_data['Genre']:
  gen.append(mp[i])
y = np.array(gen)


print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Instantiate a DecisionTreeClassifier object
dt_classifier = DecisionTreeClassifier(random_state=42, max_depth=100)
dt_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = dt_classifier.predict(X_test)

# Calculate the accuracy
accuracy_dt = accuracy_score(y_test, y_pred_dt)

print(f"Decision Tree Classifier Accuracy: {accuracy_dt:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier object
rf_classifier = RandomForestClassifier(random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_classifier.predict(X_test)

# Calculate the accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"Random Forest Classifier Accuracy: {accuracy_rf:.4f}")

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Instantiate an XGBClassifier object
xgb_classifier = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')

# Train the classifier
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_xgb = xgb_classifier.predict(X_test)

# Calculate the accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)

print(f"XGBoost Classifier Accuracy: {accuracy_xgb:.4f}")